### Estimate daily sales by using net increase of reviews for product within 24 hours

In [1]:
import csv
import numpy as np
import pandas as pd

scraped data are product title, product id, product url, product price, product rating, numbers of reviews for products, number of favorited by, store name, store sales, store url and number of reviews for store and brief reviews.

scraped data contains around 12,000 mask products, since Etsy only shows 250 pages. However, Etsy has 1,075,766 results when search for mask and 831,913 results when search for face masks, so scraped data is just a sample.


In [2]:
mask_0 = pd.read_csv("Jun10_20.csv",thousands=',')
mask_1 = pd.read_csv("Jun11_13.csv",thousands=',')

In [3]:
mask = pd.merge(mask_0[['product_id',"Num_reviews_for_product"]], mask_1,  on ='product_id',how ='inner')

In [4]:
# calculate daily change of Number of reviews for product
mask['net_reviews_for_product'] = mask['Num_reviews_for_product_y']- mask['Num_reviews_for_product_x']

In [5]:
## adjust the format of store_sales to float
#mask['store_sales'] = mask['store_sales'].str.replace(' sales','').str.replace(',','')
mask[['store_sales']]= mask[['store_sales']].apply(pd.to_numeric, errors='coerce')

In [6]:
## Estimate sales for maginal review and replace N/A by the mean
mask['estimated product sales for 1 review']=mask['store_sales']/mask['Num_reviews_for_store']
mask['estimated product sales for 1 review'].fillna(mask['estimated product sales for 1 review'].mean(),inplace=True)
print('On average, 1 review increase means  {}  sales incerase.'.format(mask['estimated product sales for 1 review'].mean()))


On average, 1 review increase means  9.256623523460313  sales incerase.


### Daily Sales and Revenue

In [7]:
daily_product_sales = mask['estimated product sales for 1 review']*mask['net_reviews_for_product']
daily_product_sales_revenue = daily_product_sales*mask['price']

In [8]:
print("Masks's daily sales is {}, and daily sales revenue is ${}."
      .format(daily_product_sales.sum().round(),daily_product_sales_revenue.sum().round(2)))

Masks's daily sales is 40785.0, and daily sales revenue is $398134.24.


#### The sample is just a proporation of total results, but it is most relevant results on Etsy.  Thus, the sample is representative.To calculate the total sales, we should use :
                            sample daily sales * No. of population / No. of sample

In [9]:
replication_rate = len(mask)*2/(len(mask_0)+len(mask_1))
replication_rate

0.8212501523090046

#### However, since the replication rate of each search is around 80%. For consumers who want face masks, their search results will also fall into the 250 pages with : 
                                     4 * 16 * 250 = 16000  products. 
 #### Thus, a margin can be applied to adjust for the calaulation.

In [10]:
margin = 16000/len(mask)
margin

1.5825914935707222

In [11]:
Daily_total_sales = daily_product_sales.sum()/margin
Daily_total_sales_revenue = daily_product_sales_revenue.sum()/margin

In [12]:
print("Masks's daily total sales is {}, and daily total sales revenue is ${}"
      .format(Daily_total_sales.round().round(),Daily_total_sales_revenue.round(2)))

Masks's daily total sales is 25771.0, and daily total sales revenue is $251571.07
